En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [76]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [77]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
from q1_time import q1_time
q1_time(file_path)

[('2021-02-18', 'neetuanjle_nitu'),
 ('2021-02-13', 'MaanDee08215437'),
 ('2021-02-17', 'RaaJVinderkaur'),
 ('2021-02-16', 'jot__b'),
 ('2021-02-20', 'MangalJ23056160'),
 ('2021-02-14', 'rebelpacifist'),
 ('2021-02-15', 'jot__b'),
 ('2021-02-23', 'Surrypuria'),
 ('2021-02-19', 'Preetm91'),
 ('2021-02-12', 'RanbirS00614606')]

In [ ]:
print(file_path)

farmers-protest-tweets-2021-2-4.json
